In [6]:
PROJECT_ID = "nasa-genie-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [2]:
! gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=sbSwYbzgdXMevvwdJm10AvlzxGGRG7&access_type=offline&code_challenge=kOF5f9ABpRFELu29AamOvXq-v4X0AyZ8h9LppJZ1OBg&code_challenge_method=S256


You are now logged in as [trishjam@google.com].
Your current project is [nasa-genie-dev].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [3]:
! gcloud config set project nasa-genie-dev

Updated property [core/project].


In [7]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [5]:
# import tempfile
# import os
from typing import List, Optional, Tuple, Dict
# from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from services.query.query_service import retrieve_references
# from services.query.query_service import rerank_references

# from common.utils.errors import (ResourceNotFoundException,
#                                  ValidationError)
# from common.utils.http_exceptions import InternalServerError
# from utils.errors import NoDocumentsIndexedException
# from google.cloud import storage
# from services import llm_generate, embeddings
# from services.query import query_prompts
# from services.query.vector_store import VectorStore
# from services.query.data_source import DataSource
# from services.query.query_service import rerank_references


# from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
#                         DEFAULT_QUERY_EMBEDDING_MODEL)
# import spacy

INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default


C:\Users\trishjam\Documents\trishjam\projects\05-nasa\code\core-solution-services2\components\llm_service\.venvt2\Lib\site-packages\langchain_community\llms\__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
C:\Users\trishjam\Documents\trishjam\projects\05-nasa\code\core-solution-services2\components\llm_service\.venvt2\Lib\site-packages\langchain_community\llms\__init__.py:329: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.

ERROR: [config/model_config.py:552 - instantiate_model_class()] Cannot load langchain model class ChatCohere


C:\Users\trishjam\Documents\trishjam\projects\05-nasa\code\core-solution-services2\components\llm_service\.venvt2\Lib\site-packages\langchain_community\llms\__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
C:\Users\trishjam\Documents\trishjam\projects\05-nasa\code\core-solution-services2\components\llm_service\.venvt2\Lib\site-packages\langchain_community\llms\__init__.py:329: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.

INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)
INFO: [utils/text_helper.py:43 - <module>()] using default spacy model
Loading default cross-enco

In [8]:
# # do this once per vm environment
# #!python -m spacy download en_core_web_sm

# nlp = spacy.load("en_core_web_sm")
# storage_client = storage.Client(project=PROJECT_ID)

## Setup

In [28]:
from common.models import QueryEngine, QueryDocument, QueryDocumentChunk

In [22]:
# from firebase
user_id = "bfrv1ifGrvAiJ9sWENHE"

# name of existing query engines
qe_name_one = "nasa-search-integrated-v2"
qe_name_one = "nasa-search-gcs"

# qe_name_two = "nasa-genie-dev-source-docs-02"

# qe_id_one = "LiHrECAO7M0pPQ1SDyjS"

# prompt_one = "Where is the Barringer Meteor Crater?"

## Understand QueryResult object model

In [23]:
qe_one = QueryEngine.find_by_name(qe_name_one)
# qe_two = QueryEngine.find_by_name(qe_name_two)

In [24]:
qe_one.id

'6kcXnzQGxUkWHTcusn5j'

In [25]:
# from services.query.query_service import retrieve_references
# from services.query.query_service import query_search

# query_refs = retrieve_references(
#     prompt_one,
#     qe_one,
#     user_id
# )
# query_refs = query_search(
#     qe_one,
#     prompt_one,
# )

# Exception: Failed to create vector extension: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "postgres"

In [26]:
query_docs = QueryDocument.find_by_query_engine_id(qe_one.id)

In [29]:
for doc in query_docs:
    print(f"[{doc.index_start}, {doc.index_end}] {doc.id} {doc.doc_url}")
    for i in range(doc.index_start, doc.index_end):
        doc_chunk = QueryDocumentChunk.find_by_index(qe_one.id, i)
        print(f"{doc_chunk.index:8} {doc_chunk.query_document_id} {doc_chunk.clean_text[0:50]}")

[19438, 19705] N6HzPqe4lnSmgjrq60a2 /b/nasa-genie-dev-ntrs/o/Orion%2Ficcfd11_predictingOrionLaunchAbortAcoustics_v2.pdf
   19438 N6HzPqe4lnSmgjrq60a2 Eleventh International Conference onComputational 
   19439 N6HzPqe4lnSmgjrq60a2 Eleventh International Conference onComputational 
   19440 N6HzPqe4lnSmgjrq60a2 The primary objectives are to establish bestpracti
   19441 N6HzPqe4lnSmgjrq60a2 Excellent agreement is obtained between CFD predic
   19442 N6HzPqe4lnSmgjrq60a2 A simplied (static and at rest)simulation of a pad
   19443 N6HzPqe4lnSmgjrq60a2 Simulation results for a supersonic ascent abortig
   19444 N6HzPqe4lnSmgjrq60a2 Overall,the CFD results demonstrate that scale-res
   19445 N6HzPqe4lnSmgjrq60a2 Orionwill be propelled into space by the powerful 
   19446 N6HzPqe4lnSmgjrq60a2 The launch abort system(LAS), detailed in Figure 2
   19447 N6HzPqe4lnSmgjrq60a2 The LAS consists of a tower where three rocket mot
   19448 N6HzPqe4lnSmgjrq60a2 Together, the crew module and theLAS for

KeyboardInterrupt: 

In [30]:
def get_chunks(qe_id):
    doc_chunks = []
    query_docs = QueryDocument.find_by_query_engine_id(qe_id)
    for doc in query_docs:
        for i in range(doc.index_start, doc.index_end):
            doc_chunk = QueryDocumentChunk.find_by_index(qe_id, i)
            doc_chunks.append(doc_chunk)
        break
    return doc_chunks

## Reranking

In [1]:
from rerankers import Reranker
from bs4 import BeautifulSoup

c:\Users\trishjam\Documents\trishjam\projects\05-nasa\code\core-solution-services2\components\llm_service\.venv_llm3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
chunks = get_chunks(qe_one.id)
print(f"Got {len(chunks)} chunks")

Got 267 chunks


In [32]:
ENCODER_MODELS = {
    "cross-encoder": Reranker('cross-encoder', verbose=0),
    "colbert": Reranker('colbert', verbose=0)
}

Loading default cross-encoder model for language en
If your model is NOT intended to be ran as a one-label cross-encoder, please reload it and specify the model_type! Otherwise, you may ignore this warning. You may specify `model_type='cross-encoder'` to suppress this warning in the future.
Loading TransformerRanker model mixedbread-ai/mxbai-rerank-base-v1
Loading default colbert model for language en
Loading ColBERTRanker model colbert-ir/colbertv2.0


In [44]:
def rank_results(encoder, prompt, docs, doc_ids):
    ranker = ENCODER_MODELS[encoder]
    results = ranker.rank(query=prompt, docs=docs, doc_ids=doc_ids)
    print(f"Got {len(results.results)} results")
    return results

def show(results, top_k=10):
    for r in results.top_k(top_k):
        print(f"{r.doc_id}  {BeautifulSoup(r.text, 'html.parser').text}")
        # print(BeautifulSoup(r.text, "html.parser").text)

In [36]:
prompt = "What is Artemis?"

In [35]:
num_chunks = 20
small_chunks = chunks[0:10]
chunk_text = [c.clean_text for c in small_chunks][:num_chunks]
chunk_ids = [c.index for c in small_chunks][:num_chunks]

In [37]:
%%time
res_ce = rank_results("cross-encoder", prompt, chunk_text, chunk_ids)

Got 10 results
CPU times: total: 18.8 s
Wall time: 6.22 s


In [45]:
show(res_ce)

19444  Overall,the CFD results demonstrate that scale-resolving simulation methodologies like the one presentedcan help characterize full-scale launch abort vehicle near-eld noise and can complement windtunnel, ground and ight tests by providing trends and reducing uncertainty for scenarios thatcannot or are not planned to be tested.Keywords: Computational Fluid Dynamics, Aero-acoustics.1 IntroductionThe National Aeronautics and Space Administrations (NASA) Artemis missions plan to bring humans tothe Moon  and eventually Mars  and return them to Earth safely onboard the Orion spacecraft.  Orionwill be propelled into space by the powerful Space Launch System, shown in Figure 1 with the Orion crewmodule and its launch abort system  the focus of this work  outlined in red.  The launch abort system(LAS), detailed in Figure 2 is attached to the top of the Orion crew module during launch and is jettisonedonce the spacecraft safely reaches the edge of space. 
19445  Orionwill be propelled int

In [40]:
%%time
res_col = rank_results("colbert", prompt, chunk_text, chunk_ids)

Got 10 results
CPU times: total: 17.3 s
Wall time: 4.51 s


In [41]:
show(res_col)

19442  A simplied (static and at rest)simulation of a pad abort ight test shows good agreement with the igh
19444  Overall,the CFD results demonstrate that scale-resolving simulation methodologies like the one prese
19446  The launch abort system(LAS), detailed in Figure 2 is attached to the top of the Orion crew module d
19443  Simulation results for a supersonic ascent abortighttestareinreasonableagreementwiththemeasurements,
19447  The LAS consists of a tower where three rocket motorsare located, and a fairing assembly which prote
19445  Orionwill be propelled into space by the powerful Space Launch System, shown in Figure 1 with the Or
19441  Excellent agreement is obtained between CFD predictionsby the Cartesian adaptive mesh renement (AMR)
19440  The primary objectives are to establish bestpractices and to validate CFD predictions for surface uc
19439  Eleventh International Conference onComputational Fluid Dynamics (ICCFD11),Maui, HI, USA, July 11-15
19438  Eleventh Internationa

## Build QueryReferences

In [7]:
query_result = QueryResult.find_by_id("0O5cJA8xJRXcSobdsZcJ")
query_ref_ids = query_result.query_refs

In [8]:
query_ref_ids

['mquvwCOQH4yo0cd9Bbgj',
 'XmsoNS0BmvuWbO0IL72A',
 'jwfQZ1QqV4yCw29VNUuW',
 'w5JytcFfTEwmF5qYqmjL',
 'b3mrzawgGSBKUaczPFKS']

In [9]:
query_ref_text = []
query_refs = []

for query_ref_id in query_ref_ids:
    query_ref = QueryReference.find_by_id(query_ref_id)
    query_doc_chunk = QueryDocumentChunk.find_by_id(query_ref.chunk_id)
    print(query_ref.id, query_ref_id, query_ref.chunk_id, query_doc_chunk.id)
    query_ref_text.append(query_doc_chunk.clean_text)
    query_refs.append(query_ref)


mquvwCOQH4yo0cd9Bbgj mquvwCOQH4yo0cd9Bbgj 91tfvDsNqB0NM7dGwQoF 91tfvDsNqB0NM7dGwQoF
XmsoNS0BmvuWbO0IL72A XmsoNS0BmvuWbO0IL72A 5qZ6cn0BKoZjQ0PpY2PN 5qZ6cn0BKoZjQ0PpY2PN
jwfQZ1QqV4yCw29VNUuW jwfQZ1QqV4yCw29VNUuW 7EDY1aXIxXeOk7I3L1tu 7EDY1aXIxXeOk7I3L1tu
w5JytcFfTEwmF5qYqmjL w5JytcFfTEwmF5qYqmjL hsgqiuKlKSyRYOnKtb4B hsgqiuKlKSyRYOnKtb4B
b3mrzawgGSBKUaczPFKS b3mrzawgGSBKUaczPFKS cl8ztT6HKFLyowBaIveR cl8ztT6HKFLyowBaIveR


## Try reranking

In [179]:
RERANK_MODEL_NAME = "cross-encoder"
reranker = Reranker(RERANK_MODEL_NAME, verbose=0)

Loading default cross-encoder model for language en
If your model is NOT intended to be ran as a one-label cross-encoder, please reload it and specify the model_type! Otherwise, you may ignore this warning. You may specify `model_type='cross-encoder'` to suppress this warning in the future.
Loading TransformerRanker model mixedbread-ai/mxbai-rerank-base-v1


In [201]:
ranked_results = reranker.rank(query=prompt_one, docs=query_ref_text, doc_ids=query_ref_ids)

In [202]:
ranked_query_ref_ids = [ r.doc_id for r in ranked_results.results ]
ranked_query_ref_ids

['b3mrzawgGSBKUaczPFKS',
 'w5JytcFfTEwmF5qYqmjL',
 'mquvwCOQH4yo0cd9Bbgj',
 'XmsoNS0BmvuWbO0IL72A',
 'jwfQZ1QqV4yCw29VNUuW']

In [203]:
sort_order_dict = {x: i for i, x in enumerate(ranked_query_ref_ids)}
sort_order_dict

{'b3mrzawgGSBKUaczPFKS': 0,
 'w5JytcFfTEwmF5qYqmjL': 1,
 'mquvwCOQH4yo0cd9Bbgj': 2,
 'XmsoNS0BmvuWbO0IL72A': 3,
 'jwfQZ1QqV4yCw29VNUuW': 4}

In [209]:
sort_order_list = [(qr, sort_order_dict[qr.id]) for qr in query_refs]
sort_order_list
sort_order_list.sort(key=lambda x: x[1])
sort_order_list

ranked_query_refs = [qr for qr, i in sort_order_list]

print([qr.id for qr in ranked_query_refs])

['b3mrzawgGSBKUaczPFKS', 'w5JytcFfTEwmF5qYqmjL', 'mquvwCOQH4yo0cd9Bbgj', 'XmsoNS0BmvuWbO0IL72A', 'jwfQZ1QqV4yCw29VNUuW']


In [29]:
def rank_references2(prompt: str,
                     query_references: List[QueryReference]) -> List[QueryReference]:

    query_ref_text = []
    query_ref_ids = []

    for query_ref in query_references:
        # query_ref = QueryReference.find_by_id(query_ref_id)
        query_doc_chunk = QueryDocumentChunk.find_by_id(query_ref.chunk_id)
        # print(query_ref.id, query_ref_id, query_ref.chunk_id, query_doc_chunk.id)
        query_ref_text.append(query_doc_chunk.clean_text)
        query_ref_ids.append(query_ref.id)

    ranked_results = reranker.rank(query=prompt, docs=query_ref_text, doc_ids=query_ref_ids)
    # putting the references list in order by rank
    ranked_query_ref_ids = [ r.doc_id for r in ranked_results.results ]
    sort_dict = {x: i for i, x in enumerate(ranked_query_ref_ids)}
    sort_list = [(qr, sort_dict[qr.id]) for qr in query_refs]
    sort_list.sort(key=lambda x: x[1])

    # just return the QueryReferences
    ranked_query_refs = [qr for qr, i in sort_order_list]

    return ranked_query_refs

In [220]:
query_result = QueryResult.find_by_id("0O5cJA8xJRXcSobdsZcJ")
query_ref_ids = query_result.query_refs

query_refs = []
for query_ref_id in query_ref_ids:
    print(query_ref_id)
    query_refs.append(QueryReference.find_by_id(query_ref_id))

ranked_query_refs = rank_references2(prompt_one, query_refs)

In [221]:
for qr in ranked_query_refs:
    print(qr.id)

b3mrzawgGSBKUaczPFKS
w5JytcFfTEwmF5qYqmjL
mquvwCOQH4yo0cd9Bbgj
XmsoNS0BmvuWbO0IL72A
jwfQZ1QqV4yCw29VNUuW


## Test reranking function

In [10]:
from services.query.query_service import rerank_references

In [11]:
ranked_query_refs = rerank_references(prompt_one, query_refs)

INFO: [query/query_service.py:272 - rerank_references()] Reranking 5 references for query_prompt=[Where is the Barringer Meteor Crater?]


In [12]:
for r in ranked_query_refs:
    print(r.id, r.chunk_id)

b3mrzawgGSBKUaczPFKS cl8ztT6HKFLyowBaIveR
w5JytcFfTEwmF5qYqmjL hsgqiuKlKSyRYOnKtb4B
mquvwCOQH4yo0cd9Bbgj 91tfvDsNqB0NM7dGwQoF
XmsoNS0BmvuWbO0IL72A 5qZ6cn0BKoZjQ0PpY2PN
jwfQZ1QqV4yCw29VNUuW 7EDY1aXIxXeOk7I3L1tu
